# Machine Learning vs Las Vegas

## Analyzing College Basketball Results Past to Model the Future

### Sean Hoffman - Final Project - DU-DEN-DATA-PT-11-2019-U-C

#### Dependencies

In [2]:
import numpy as np
import pandas as pd

# sportsreference is a is a free python API that pulls the stats from www.sports-reference.com and 
# allows them to be easily be used in python-based applications.
# See https://github.com/roclark/sportsreference for details.
from sportsreference.ncaab.teams import Teams

# tqdm is a smart progress meter for python to allow you to see progress being made while large loops are running.
# See https://github.com/tqdm/tqdm for details.
from tqdm import tqdm

In [10]:
i = 0
for game in sample_team.schedule:
    i+=1
    if i > 1:
        continue
    else:
        sample_bs = game.boxscore
        
dir(sample_bs)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_away_assist_percentage',
 '_away_assists',
 '_away_block_percentage',
 '_away_blocks',
 '_away_defensive_rating',
 '_away_defensive_rebound_percentage',
 '_away_defensive_rebounds',
 '_away_effective_field_goal_percentage',
 '_away_field_goal_attempts',
 '_away_field_goal_percentage',
 '_away_field_goals',
 '_away_free_throw_attempt_rate',
 '_away_free_throw_attempts',
 '_away_free_throw_percentage',
 '_away_free_throws',
 '_away_minutes_played',
 '_away_name',
 '_away_offensive_rating',
 '_away_offensive_rebound_percentage',
 '_away_offensive_rebounds',
 '_away_personal_fouls',
 '_away_players',
 '_away_points',
 '_awa

In [17]:
print(sample_bs.dataframe.shape)

(1, 86)


In [18]:
print(list(sample_bs.dataframe))

['away_assist_percentage', 'away_assists', 'away_block_percentage', 'away_blocks', 'away_defensive_rating', 'away_defensive_rebound_percentage', 'away_defensive_rebounds', 'away_effective_field_goal_percentage', 'away_field_goal_attempts', 'away_field_goal_percentage', 'away_field_goals', 'away_free_throw_attempt_rate', 'away_free_throw_attempts', 'away_free_throw_percentage', 'away_free_throws', 'away_losses', 'away_minutes_played', 'away_offensive_rating', 'away_offensive_rebound_percentage', 'away_offensive_rebounds', 'away_personal_fouls', 'away_points', 'away_ranking', 'away_steal_percentage', 'away_steals', 'away_three_point_attempt_rate', 'away_three_point_field_goal_attempts', 'away_three_point_field_goal_percentage', 'away_three_point_field_goals', 'away_total_rebound_percentage', 'away_total_rebounds', 'away_true_shooting_percentage', 'away_turnover_percentage', 'away_turnovers', 'away_two_point_field_goal_attempts', 'away_two_point_field_goal_percentage', 'away_two_point_fie

In [11]:
sample_bs.home_players
sample_player = sample_bs.home_players[0]

In [12]:
print(list(dir(sample_player)))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_assist_percentage', '_assists', '_block_percentage', '_blocks', '_defensive_rating', '_defensive_rebound_percentage', '_defensive_rebounds', '_effective_field_goal_percentage', '_field_goal_attempts', '_field_goal_percentage', '_field_goals', '_free_throw_attempt_rate', '_free_throw_attempts', '_free_throw_percentage', '_free_throws', '_index', '_minutes_played', '_name', '_offensive_rating', '_offensive_rebound_percentage', '_offensive_rebounds', '_parse_player_data', '_parse_value', '_personal_fouls', '_player_data', '_player_id', '_points', '_steal_percentage', '_steals', '_three_point_attempt_rate', '_three_point_attempts', '_three_point_per

#### Create a list of boxscore URLS for games played in each season using 'try/except' because not all teams have information available for every season

In [3]:
seasons = range(2011, 2020)

master_array = []

for season in tqdm(seasons):
    try:
        season_team_list = Teams(str(season))
        for team in season_team_list:
            team_schedule = team.schedule
            for game in team_schedule:
                boxscore = game.boxscore_index
                master_array.append([str(season),team.name,boxscore])
    except:
        continue

schedule_df = pd.DataFrame(master_array, columns=
                          ['Season','TeamName','BoxScoreIndex'])

schedule_df.to_csv('./output/schedule.csv',index=None)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [24:21<00:00, 146.18s/it]


In [14]:
print(len(schedule_df))

113381


In [16]:
schedule_df.groupby('Season')['BoxScoreIndex'].count()

Season
2010    10650
2011    11135
2012    11145
2013    11281
2014    11455
2015    11431
2016    11468
2017    11511
2018    11542
2019    11652
Name: BoxScoreIndex, dtype: int64

In [21]:
print(schedule_df.iloc[0].BoxScoreIndex)

2009-11-13-air-force


In [22]:
print(schedule_df.dtypes)

Season            int64
TeamName         object
BoxScoreIndex    object
dtype: object


#### Import each of the available game boxscores from sports-reference.com for each season by looping through API calls. Assemble the data in csv files for each season.

In [26]:
from sportsreference.ncaab.boxscore import Boxscore

season = 2011
box_df = None

schedule_df = pd.read_csv('./output/schedule.csv')

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11135it [2:00:13,  1.54it/s]


In [3]:
# Re-read schedule.csv into schedule_df due to cleaning out exceptions

schedule_df = pd.read_csv('./output/schedule.csv')

In [29]:
season = 2012
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11144it [1:58:37,  1.57it/s]


In [30]:
season = 2013
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11281it [2:01:16,  1.55it/s]


In [32]:
season = 2014
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11455it [2:01:00,  1.58it/s]


In [4]:
from sportsreference.ncaab.boxscore import Boxscore

season = 2015
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11430it [1:32:33,  2.06it/s]


In [8]:
from sportsreference.ncaab.boxscore import Boxscore

season = 2016
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11468it [1:40:01,  1.91it/s]


In [9]:
from sportsreference.ncaab.boxscore import Boxscore

season = 2017
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11511it [1:42:29,  1.87it/s]


In [10]:
from sportsreference.ncaab.boxscore import Boxscore

season = 2018
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11544it [1:44:00,  1.85it/s]


In [12]:
from sportsreference.ncaab.boxscore import Boxscore

season = 2019
box_df = None

season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxScoreIndex']
    _df = Boxscore(box_link).dataframe
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
        
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)

11652it [2:02:28,  1.59it/s]
